# Main Idea

The main idea here is to identify **normal periods** (negative samples) and **acute hypotensive episodes** (positive samples). The plan is to use lab tests/measurements taken from several hours to half an hour before these periods (this window can be adjusted later for optimal performance) as features for prediction.

## Key Measurements
The key measurements include:
- **Diastolic blood pressure** (220051)
- **Systolic blood pressure** (220050)
- **Heart rate** (220045)
- **SpO2** (220277)
- **MAP (Mean Arterial Pressure)** (220052)
- **Respiratory rate** (220210)

Additionally, some general features like **age**, **gender**, etc., will be included to form the dataset for training and prediction.

## Data Source and Processing
My data primarily comes from executing **SQL queries on Google BigQuery**. After that, the corresponding query results are downloaded as **CSV files**, which are then read into my code for use. If needed, we can modify the workflow to directly access the database and execute SQL queries in **Google Colab** to streamline the process later.


In [1]:
from IPython.display import display

import warnings

warnings.filterwarnings('ignore')

# Data Cleaning



- Removed outliers from the dataset using Tukey's method and Modified Z-score method.
Replaced outliers with NaN and then dropped them from the dataset.

In [2]:
#clean the "all map for patient having less than 60.csv" (used to extract samples) by removing outliers directly.

import numpy as np
import pandas as pd

# Tukey method to detect and replace outliers
def detect_outliers_tukey(df, value_column):
    Q1 = df[value_column].quantile(0.25)
    Q3 = df[value_column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Detect outliers
    outliers = (df[value_column] < lower_bound) | (df[value_column] > upper_bound)

    # Replace outliers with NaN
    df.loc[outliers, value_column] = np.nan
    return df

# Modified Z-score method to detect and replace outliers
def detect_outliers_z(df, value_column):
    median = df[value_column].median()
    mad = np.median(np.abs(df[value_column] - median))
    modified_z_score = 0.6745 * (df[value_column] - median) / mad

    # Detect outliers
    outliers = np.abs(modified_z_score) > 3.5

    # Replace outliers with NaN
    df.loc[outliers, value_column] = np.nan
    return df

# Detect and remove outliers using both methods
def clean_outliers(df, value_column='valuenum'):
    # First, use the Tukey method
    df = detect_outliers_tukey(df, value_column)
    # Then, use the modified Z-score method
    df = detect_outliers_z(df, value_column)
    # Remove rows with NaN values
    df = df.dropna(subset=[value_column])
    return df

# Load data
df = pd.read_csv('all map for patient having less than 60.csv')

# Clean outliers in the 'valuenum' column before filtering
df = clean_outliers(df, value_column='valuenum')

# Extracting Positive Samples (Acute Hypotensive Episodes)


- Identified periods where patients experienced acute hypotensive episodes based on specific criteria.
- Filtered the dataset to include only these periods.

In [3]:
#extract positive samples(periods of acute hypotensive episodes)
#The criteria for selection are: if there are two consecutive records（labtest） with values less than 60 mmHg, and the time difference between them is greater than 30 minutes, then this period is considered as an acute hypotensive episode.
#Additionally, I need to check whether the record（labtest） prior to these two records were greater than 60 mmHg to ensure that the hypotensive episode started precisely at this period, and it's not part of an ongoing event.

# Sort the dataframe by subject_id, hadm_id, stay_id, and charttime
df.sort_values(by=['subject_id', 'hadm_id','stay_id', 'charttime'], inplace=True)

# Convert charttime to datetime format
df['charttime'] = pd.to_datetime(df['charttime'])

# Calculate the time difference between the current and the next record, and create a new column for time difference
df['time_diff_next'] = df.groupby(['subject_id', 'hadm_id','stay_id'])['charttime'].shift(-1) - df['charttime']
df['time_diff'] = df.groupby(['subject_id', 'hadm_id','stay_id'])['charttime'].diff()

# Columns that need to be checked for identical values
columns_to_check = ['subject_id', 'hadm_id', 'stay_id']

# Filtering condition
condition = (
    ((df['valuenum'] < 60) &  # The current value is less than 60
    (df['valuenum'].shift(-1) < 60) &  # The next value is also less than 60
    (df['valuenum'].rolling(window=2, min_periods=2).min().shift(1) >= 60) &  # The minimum value in the 2-window rolling period before is greater than or equal to 60
    (df['time_diff_next'] > pd.Timedelta(minutes=30)) &  # The time difference between the current and next record is greater than 30 minutes
    (df['time_diff_next'] < pd.Timedelta(hours=12)) &  # The time difference between the current and next record is less than 12 hours
    ((df['charttime'] - df.groupby(['subject_id', 'hadm_id','stay_id'])['charttime'].shift(2)) >= pd.Timedelta(hours=1)) &  # Time difference from 2 records before is greater than or equal to 1 hour
    ((df['charttime'] - df.groupby(['subject_id', 'hadm_id','stay_id'])['charttime'].shift(2)) <= pd.Timedelta(hours=24)) &  # Time difference from 2 records before is less than or equal to 24 hours
    (df[columns_to_check].shift(1) == df[columns_to_check]).all(axis=1) &  # Ensure the values for subject_id, hadm_id, stay_id remain the same in previous records
    (df[columns_to_check].shift(2) == df[columns_to_check]).all(axis=1) &  # Ensure the values for subject_id, hadm_id, stay_id remain the same 2 records back
    (df[columns_to_check].shift(-1) == df[columns_to_check]).all(axis=1))   # Ensure the values for subject_id, hadm_id, stay_id remain the same in the next record
)

# Filter the dataframe based on the above condition
filtered_df_1 = df[condition]

# Display the filtered result
print(filtered_df_1)

# Calculate the number of unique combinations of subject_id and hadm_id
unique_combinations_count = filtered_df_1.groupby(['subject_id', 'hadm_id']).ngroups

# Display the result
#print(f"Number of unique subject_id + hadm_id combinations: {unique_combinations_count}")
display(len(filtered_df_1))

         subject_id   hadm_id   stay_id  caregiver_id           charttime  \
730        10004401  29988601  32773003         38908 2144-01-27 20:02:00   
735        10004401  29988601  32773003         38908 2144-01-28 01:00:00   
776        10004401  29988601  32773003         83221 2144-01-29 19:00:00   
791        10004401  29988601  32773003         91879 2144-01-30 10:00:00   
804        10004401  29988601  32773003         24745 2144-01-30 21:00:00   
...             ...       ...       ...           ...                 ...   
1791605    19986880  28386154  32959861         36239 2185-08-03 08:00:00   
1791632    19986880  28386154  32959861         36239 2185-08-04 01:00:00   
1791704    19986880  28386154  32959861         29373 2185-08-06 16:00:00   
1791950    19989783  26984195  32761676         26019 2130-07-13 20:00:00   
1792945    19993726  20161163  31439824         12665 2166-09-20 21:00:00   

                   storetime  itemid  value  valuenum valueuom  warning  \


10719

In [4]:
#Simply check if the selected data is what I want

i = df.index[df['charttime'] == '2144-01-27 20:02:00'].tolist()[0]


previous_row = df.iloc[i-3] if i > 0 else None
current_row = df.iloc[i]
next_row = df.iloc[i+1] if i < len(df)-1 else None


display("previous record:")
display(previous_row)

display("\n current:")
display(current_row)

display("\n next record:")
display(next_row)

'previous record:'

,742
subject_id,10004401
hadm_id,29988601
stay_id,32773003
caregiver_id,91879
charttime,2144-01-28 09:00:00
storetime,2144-01-28 09:00:00
itemid,220052
value,49.0
valuenum,49.0
valueuom,mmHg


'\n current:'

,745
subject_id,10004401
hadm_id,29988601
stay_id,32773003
caregiver_id,91879
charttime,2144-01-28 12:00:00
storetime,2144-01-28 12:37:00
itemid,220052
value,53.0
valuenum,53.0
valueuom,mmHg


'\n next record:'

,746
subject_id,10004401
hadm_id,29988601
stay_id,32773003
caregiver_id,91879
charttime,2144-01-28 13:00:00
storetime,2144-01-28 13:45:00
itemid,220052
value,52.0
valuenum,52.0
valueuom,mmHg


# Extracting Negative Samples (Normal Periods)


- Identified normal periods where measurements were consistently above 60 mmHg.
- Filtered the dataset to include only these normal periods.
- Selected only one sample per day to reduce the number of negative samples.

In [5]:
# extract negative samples(normal period)
#If a record, including the previous and subsequent records, has values all greater than 60 mmHg, we can consider this period as a normal period.

condition = (
    (df['valuenum'] >= 60) &
    (df['valuenum'].rolling(window=3, min_periods=3).min().shift(-1) >= 60) &
    (df['valuenum'].rolling(window=3, min_periods=3).min().shift(1) >= 60) &
    ((df['charttime'] - df.groupby(['subject_id', 'hadm_id','stay_id'])['charttime'].shift(3)) >= pd.Timedelta(hours=1.5)) &
    ((df['charttime'] - df.groupby(['subject_id', 'hadm_id','stay_id'])['charttime'].shift(3)) <= pd.Timedelta(hours=12)) &
    ((df.groupby(['subject_id', 'hadm_id','stay_id'])['charttime'].shift(-3))-df['charttime'] >= pd.Timedelta(hours=1.5)) &
    ((df.groupby(['subject_id', 'hadm_id','stay_id'])['charttime'].shift(-3))-df['charttime'] <= pd.Timedelta(hours=12)) &
    (df[columns_to_check].shift(1) == df[columns_to_check]).all(axis=1) &
    (df[columns_to_check].shift(2) == df[columns_to_check]).all(axis=1) &
    (df[columns_to_check].shift(3) == df[columns_to_check]).all(axis=1) &
    (df[columns_to_check].shift(-1) == df[columns_to_check]).all(axis=1) &
    (df[columns_to_check].shift(-2) == df[columns_to_check]).all(axis=1) &
    (df[columns_to_check].shift(-3) == df[columns_to_check]).all(axis=1)

)


filtered_df_0 = df[condition]


print(filtered_df_0)


unique_combinations_count = filtered_df_0.groupby(['subject_id', 'hadm_id']).ngroups


#print(f"Number of unique subject_id + hadm_id combinations:: {unique_combinations_count}")
print(len(filtered_df_0))

         subject_id   hadm_id   stay_id  caregiver_id           charttime  \
3          10002013  23581541  39060235         65197 2160-05-18 17:00:00   
4          10002013  23581541  39060235         65197 2160-05-18 18:00:00   
5          10002013  23581541  39060235         65197 2160-05-18 19:00:00   
6          10002013  23581541  39060235          6768 2160-05-18 20:00:00   
7          10002013  23581541  39060235          6768 2160-05-18 21:00:00   
...             ...       ...       ...           ...                 ...   
1794132    19999442  26785317  32336619         53164 2148-11-20 16:00:00   
1794133    19999442  26785317  32336619         53164 2148-11-20 17:00:00   
1794134    19999442  26785317  32336619         53164 2148-11-20 18:04:00   
1794135    19999442  26785317  32336619         53164 2148-11-20 19:00:00   
1794136    19999442  26785317  32336619         59028 2148-11-20 20:00:00   

                   storetime  itemid  value  valuenum valueuom  warning  \


In [6]:
# Since there are too many negative samples (normal periods), only one sample per day will be selected
filtered_df_0['date'] = filtered_df_0['charttime'].dt.date

# Group by subject_id, hadm_id, stay_id, date and select the last record from each group
filtered_df_0_unique = filtered_df_0.groupby(['subject_id', 'hadm_id', 'stay_id', 'date']).last().reset_index()

# View the result
print(filtered_df_0_unique)

# Calculate the number of unique subject_id + hadm_id combinations
unique_combinations_count = filtered_df_0_unique.groupby(['subject_id', 'hadm_id']).ngroups
print(f"Number of unique subject_id + hadm_id combinations: {unique_combinations_count}")
print(len(filtered_df_0_unique))

       subject_id   hadm_id   stay_id        date  caregiver_id  \
0        10002013  23581541  39060235  2160-05-18          6768   
1        10002013  23581541  39060235  2160-05-19          6768   
2        10002428  23473524  35479615  2156-05-11         70245   
3        10002428  23473524  35479615  2156-05-12         53739   
4        10002428  23473524  35479615  2156-05-13         55876   
...           ...       ...       ...         ...           ...   
75138    19998843  24842066  30988867  2187-02-07         47270   
75139    19998843  24842066  30988867  2187-02-08         59028   
75140    19999287  20175828  35165301  2197-08-07          7340   
75141    19999442  26785317  32336619  2148-11-19         59028   
75142    19999442  26785317  32336619  2148-11-20         59028   

                charttime            storetime  itemid  value  valuenum  \
0     2160-05-18 23:00:00  2160-05-18 23:01:00  220052   80.0      80.0   
1     2160-05-19 02:00:00  2160-05-19 02:06:0

# Balancing the Dataset


- Added labels to the positive (1) and negative (0) samples.
- Combined the datasets and performed downsampling to create a balanced dataset with equal numbers of positive and negative samples.

In [7]:
# Add label to the samples
filtered_df_1['label'] = 1
filtered_df_0_unique['label'] = 0

# Select the required columns
columns_to_keep = ['subject_id', 'hadm_id', 'stay_id', 'caregiver_id', 'charttime', 'label']

# Concatenate the two DataFrames by columns
combined_df = pd.concat([filtered_df_1[columns_to_keep], filtered_df_0_unique[columns_to_keep]])

# Reset the index
combined_df.reset_index(drop=True, inplace=True)

# View the combined result
combined_df

,subject_id,hadm_id,stay_id,caregiver_id,charttime,label
0,10004401,29988601,32773003,38908,2144-01-27 20:02:00,1
1,10004401,29988601,32773003,38908,2144-01-28 01:00:00,1
2,10004401,29988601,32773003,83221,2144-01-29 19:00:00,1
3,10004401,29988601,32773003,91879,2144-01-30 10:00:00,1
4,10004401,29988601,32773003,24745,2144-01-30 21:00:00,1
...,...,...,...,...,...,...
85857,19998843,24842066,30988867,47270,2187-02-07 23:00:00,0
85858,19998843,24842066,30988867,59028,2187-02-08 15:00:00,0
85859,19999287,20175828,35165301,7340,2197-08-07 04:00:00,0
85860,19999442,26785317,32336619,59028,2148-11-19 23:04:00,0


In [8]:
# Negative samples still far outnumber positive samples, so we perform downsampling
# To create balanced dataset with 8,000 samples (4,000 from each class)

# Sample 4,000 samples from the data with label 0
df_label_0 = combined_df[combined_df['label'] == 0].sample(n=4000, random_state=42)

# Sample 4,000 samples from the data with label 1
df_label_1 = combined_df[combined_df['label'] == 1].sample(n=4000, random_state=42)

# Concatenate the two sampled datasets
combined_sampled_df = pd.concat([df_label_0, df_label_1])

# Reset the index
combined_sampled_df.reset_index(drop=True, inplace=True)

# View the result
print("Combined sampled dataset shape:", combined_sampled_df.shape)
combined_sampled_df.head()

Combined sampled dataset shape: (8000, 6)


,subject_id,hadm_id,stay_id,caregiver_id,charttime,label
0,12765745,24692465,38629592,21810,2147-07-20 23:00:00,0
1,19345174,24168361,32217390,1867,2178-03-11 20:01:00,0
2,15782910,24816970,39146812,12665,2177-06-15 23:00:00,0
3,10545967,27498545,32767451,55621,2186-05-08 23:00:00,0
4,19458874,27624253,37193919,36239,2149-10-25 23:00:00,0


# Cleaning Feature Data (item3.csv)


- Cleaned the item3.csv file by detecting and handling outliers.
- Replaced outliers using interpolation for continuous data.
- Saved the cleaned data to new files for future use.


In [9]:
import numpy as np
import pandas as pd

# Function to detect and replace outliers using Tukey's method
def detect_outliers_tukey(df, value_column):
   Q1 = df[value_column].quantile(0.25)
   Q3 = df[value_column].quantile(0.75)
   IQR = Q3 - Q1
   lower_bound = Q1 - 1.5 * IQR
   upper_bound = Q3 + 1.5 * IQR

   # Detect outliers
   outliers = (df[value_column] < lower_bound) | (df[value_column] > upper_bound)

   # Replace outliers with NaN
   df.loc[outliers, value_column] = np.nan
   return df

# Function to detect and replace outliers using Modified Z-score method
def detect_outliers_z(df, value_column):
   median = df[value_column].median()
   mad = np.median(np.abs(df[value_column] - median))
   modified_z_score = 0.6745 * (df[value_column] - median) / mad

   # Detect outliers
   outliers = np.abs(modified_z_score) > 3.5

   # Replace outliers with NaN
   df.loc[outliers, value_column] = np.nan
   return df

# Group by 'itemid' to handle different measurements separately
def clean_outliers_by_itemid(df, value_column='valuenum'):
   df_cleaned = pd.DataFrame()

   # Process each itemid separately
   for itemid, group in df.groupby('itemid'):
       # Detect outliers using Tukey's method and Modified Z-score method
       group = detect_outliers_tukey(group, value_column)
       group = detect_outliers_z(group, value_column)

       # Fill NaN values (outliers) using linear interpolation
       group[value_column] = group[value_column].interpolate(method='linear')

       # Append cleaned group
       df_cleaned = pd.concat([df_cleaned, group], axis=0)
       print("1")

   return df_cleaned

# Example usage:
# Load your dataset
items_df = pd.read_csv('item3.csv')

# Clean outliers in your dataset
items_df_cleaned = clean_outliers_by_itemid(items_df)

# Save the cleaned data to a new file if needed
items_df_cleaned.to_csv('cleaned_item3.csv', index=False)

1
1
1
1
1
1


In [10]:
print(items_df.shape)
print(items_df_cleaned.shape)

(13569907, 6)
(13569907, 6)



*   Another way which drop outliers directly.



In [11]:
#import numpy as np
#import pandas as pd
#
## Function to detect and replace outliers using Tukey's method
#def detect_outliers_tukey(df, value_column):
#    Q1 = df[value_column].quantile(0.25)
#    Q3 = df[value_column].quantile(0.75)
#    IQR = Q3 - Q1
#    lower_bound = Q1 - 1.5 * IQR
#    upper_bound = Q3 + 1.5 * IQR
#
#    # Detect outliers
#    outliers = (df[value_column] < lower_bound) | (df[value_column] > upper_bound)
#
#    # Replace outliers with NaN
#    df.loc[outliers, value_column] = np.nan
#    return df
#
## Function to detect and replace outliers using Modified Z-score method
#def detect_outliers_z(df, value_column):
#    median = df[value_column].median()
#    mad = np.median(np.abs(df[value_column] - median))
#    modified_z_score = 0.6745 * (df[value_column] - median) / mad
#
#    # Detect outliers
#    outliers = np.abs(modified_z_score) > 3.5
#
#    # Replace outliers with NaN
#    df.loc[outliers, value_column] = np.nan
#    return df
#
## Group by 'itemid' to handle different measurements separately
#def clean_outliers_by_itemid(df, value_column='valuenum'):
#    df_cleaned = pd.DataFrame()
#
#    # Process each itemid separately
#    for itemid, group in df.groupby('itemid'):
#        # Detect outliers using Tukey's method and Modified Z-score method
#        group = detect_outliers_tukey(group, value_column)
#        group = detect_outliers_z(group, value_column)
#
#        # Drop rows with NaN values (outliers)
#        group = group.dropna(subset=[value_column])
#
#        # Append cleaned group
#        df_cleaned = pd.concat([df_cleaned, group], axis=0)
#
#    return df_cleaned
#
## Example usage:
## Load your dataset
#items_df = pd.read_csv('item3.csv')
#
## Clean outliers in your dataset
#items_df_cleaned = clean_outliers_by_itemid(items_df)
#
## Save the cleaned data to a new file if needed
#items_df_cleaned.to_csv('cleaned_item3_2.csv', index=False)
#

# Feature Extraction


- Extracted statistical features (mean, max, min, median, etc.) for each measurement within a specified time window (from x.5 to 0.5 hours before the sample time).
- Calculated 12 features consistent with the method used in the reference paper.
- Stored the features in a new DataFrame for merging.

In [12]:
 from scipy.stats import skew, kurtosis

 items_df = pd.read_csv('cleaned_item3.csv')

 # First, convert charttime to datetime format
 combined_sampled_df['charttime'] = pd.to_datetime(combined_sampled_df['charttime'])
 items_df['charttime'] = pd.to_datetime(items_df['charttime'])

 # Create an empty list to store the results
 new_features = []
 i = 0

 # Iterate through each row in combined_df
 for _, row in combined_sampled_df.iterrows():
     subject_id = row['subject_id']
     hadm_id = row['hadm_id']
     stay_id = row['stay_id']
     charttime = row['charttime']

     # Find matching records in items_df based on subject_id, hadm_id, and stay_id
     relevant_items = items_df[
         (items_df['subject_id'] == subject_id) &
         (items_df['hadm_id'] == hadm_id) &
         (items_df['stay_id'] == stay_id)
     ]

     # Filter events within the time window (from 0.5 hours to x.5 hours before the current record)
     time_window_items = relevant_items[
         (relevant_items['charttime'] >= charttime - pd.Timedelta(hours=5.5)) &
         (relevant_items['charttime'] <= charttime - pd.Timedelta(hours=0.5))
     ]

     # Since multiple measurements may be taken within this time window, we choose to use statistics such as the mean, max, min, etc. (a total of 12 features),
     # This 12 features consistent with the method used in the reference paper.
     # Group by itemid and calculate statistics for valuenum (mean, max, min, etc.)
     grouped = time_window_items.groupby('itemid')['valuenum'].agg([
         'mean',                # Mean
         'max',                 # Maximum
         'min',                 # Minimum
         'median',              # Median
         'std',                 # Standard Deviation
         ('skewness', skew),    # Skewness (using scipy.stats.skew)
         ('kurtosis', kurtosis),# Kurtosis (using scipy.stats.kurtosis)
         ('q75', lambda x: np.percentile(x, 75)),  # 75th percentile
         ('q25', lambda x: np.percentile(x, 25)),  # 25th percentile
         ('mad', lambda x: np.mean(np.abs(x - np.mean(x)))),  # Mean Absolute Deviation
         ('range', lambda x: np.max(x) - np.min(x)),  # Range
         'var'                  # Variance
     ])

     # Store the result as a dictionary for easier merging with combined_df
     grouped_dict = grouped.to_dict('index')

     # Add to the new_features list to be merged later
     new_features.append(grouped_dict)

     # Print progress
     i += 1
     print(i)

 # Convert the new features to a DataFrame and merge with combined_df
 new_features_df = pd.DataFrame(new_features)
 final_df = pd.concat([combined_sampled_df.reset_index(drop=True), new_features_df.reset_index(drop=True)], axis=1)

流式输出内容被截断，只能显示最后 5000 行内容。
3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
3011
3012
3013
3014
3015
3016
3017
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3176
3177
3178
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
319

# Adding Additional Features (Vasopressin and Ventilation Usage)


- Added binary features indicating whether vasopressin or ventilation was used within the specified time window before the sample time.
- Although initial tests showed these features might not significantly affect results, they were included for completeness.

In [13]:
# Add vasopressin and ventilation usage information as binary features (1 if used, 0 if not)
# Testing revealed that these two features do not significantly affect the results, so they might be discarded.

vasopressin_df = pd.read_csv('vasopressin.csv')
ventilation_df = pd.read_csv('ventilation.csv')

vasopressin_df['starttime'] = pd.to_datetime(vasopressin_df['starttime'])
vasopressin_df['endtime'] = pd.to_datetime(vasopressin_df['endtime'])
ventilation_df['starttime'] = pd.to_datetime(ventilation_df['starttime'])
ventilation_df['endtime'] = pd.to_datetime(ventilation_df['endtime'])



# Define a function to check if vasopressin or ventilation was given in the required time window
def check_treatment(row):
    charttime = row['charttime']
    stay_id = row['stay_id']

    time_start = charttime - pd.Timedelta(hours=5.5)
    time_end = charttime - pd.Timedelta(hours=0.5)

    # Check for vasopressin
    vasopressin_given = vasopressin_df[
        (vasopressin_df['stay_id'] == stay_id) &
        (vasopressin_df['starttime'] >= time_start) &
        (vasopressin_df['starttime'] <= time_end)
    ]

    # Check for ventilation
    ventilation_given = ventilation_df[
        (ventilation_df['stay_id'] == stay_id) &
        (ventilation_df['starttime'] >= time_start) &
        (ventilation_df['starttime'] <= time_end)
    ]

    # Assign 1 if given, 0 otherwise
    row['vasopressin'] = 1 if not vasopressin_given.empty else 0
    row['ventilation'] = 1 if not ventilation_given.empty else 0

    return row

# Apply the function to every row in final_df_cleaned
final_df = final_df.apply(check_treatment, axis=1)

# Adding General Features (Age, Gender, APSIII, SOFA)


- Merged additional patient data such as age, gender, APSIII scores, and SOFA scores into the main dataset.
- This provided more context and potential predictive power for the model.

In [14]:
#add general features like age, gender...

apsiii_df = pd.read_csv('apsiii.csv')
age_df= pd.read_csv('age.csv')
gender_df= pd.read_csv('gender.csv')
sofa_df= pd.read_csv('sofa.csv')

final_df = pd.merge(final_df, apsiii_df, on=['subject_id', 'hadm_id', 'stay_id'], how='left')
final_df = pd.merge(final_df, age_df, on=['subject_id', 'hadm_id'], how='left')
final_df = pd.merge(final_df, gender_df, on=['subject_id'], how='left')
final_df = pd.merge(final_df, sofa_df, on=['subject_id', 'hadm_id', 'stay_id'], how='left')

# show results
display(final_df)

,subject_id,hadm_id,stay_id,caregiver_id,charttime,label,220045,220050,220051,220052,220210,220277,vasopressin,ventilation,apsiii,age,gender,anchor_age,SOFA
0,12765745,24692465,38629592,21810,2147-07-20 23:00:00,0,"{'mean': 60.0, 'max': 73.0, 'min': 55.0, 'medi...","{'mean': 111.0, 'max': 117.0, 'min': 100.0, 'm...","{'mean': 44.0, 'max': 57.0, 'min': 39.0, 'medi...","{'mean': 64.6, 'max': 70.0, 'min': 60.0, 'medi...","{'mean': 18.2, 'max': 19.0, 'min': 18.0, 'medi...","{'mean': 99.6, 'max': 100.0, 'min': 99.0, 'med...",0,0,43.0,83,F,83,3.0
1,19345174,24168361,32217390,1867,2178-03-11 20:01:00,0,"{'mean': 70.57142857142857, 'max': 77.0, 'min'...","{'mean': 101.0, 'max': 119.0, 'min': 90.0, 'me...","{'mean': 51.0, 'max': 57.0, 'min': 47.0, 'medi...","{'mean': 66.0, 'max': 75.0, 'min': 61.0, 'medi...","{'mean': 11.714285714285714, 'max': 18.0, 'min...","{'mean': 99.14285714285714, 'max': 100.0, 'min...",0,0,64.0,46,F,46,9.0
2,15782910,24816970,39146812,12665,2177-06-15 23:00:00,0,"{'mean': 98.0, 'max': 104.0, 'min': 84.0, 'med...","{'mean': 110.8, 'max': 125.0, 'min': 98.0, 'me...","{'mean': 59.2, 'max': 72.0, 'min': 53.0, 'medi...","{'mean': 78.4, 'max': 92.0, 'min': 70.0, 'medi...","{'mean': 23.2, 'max': 27.0, 'min': 17.0, 'medi...","{'mean': 97.4, 'max': 98.0, 'min': 97.0, 'medi...",0,0,59.0,77,M,75,6.0
3,10545967,27498545,32767451,55621,2186-05-08 23:00:00,0,"{'mean': 85.0, 'max': 95.0, 'min': 76.0, 'medi...","{'mean': 149.5, 'max': 163.0, 'min': 132.0, 'm...","{'mean': 62.6, 'max': 77.0, 'min': 56.0, 'medi...","{'mean': 91.5, 'max': 98.0, 'min': 85.0, 'medi...","{'mean': 22.0, 'max': 32.0, 'min': 17.0, 'medi...","{'mean': 93.4, 'max': 96.0, 'min': 92.0, 'medi...",0,0,32.0,61,F,61,5.0
4,19458874,27624253,37193919,36239,2149-10-25 23:00:00,0,"{'mean': 71.8, 'max': 77.0, 'min': 69.0, 'medi...","{'mean': 111.2, 'max': 121.0, 'min': 101.0, 'm...","{'mean': 59.6, 'max': 61.0, 'min': 56.0, 'medi...","{'mean': 75.6, 'max': 79.0, 'min': 73.0, 'medi...","{'mean': 14.0, 'max': 16.0, 'min': 12.0, 'medi...","{'mean': 97.8, 'max': 98.0, 'min': 97.0, 'medi...",0,0,89.0,57,M,56,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,11474034,22412399,30615140,34084,2127-09-30 02:00:00,1,"{'mean': 66.2, 'max': 96.0, 'min': 56.0, 'medi...","{'mean': 94.75, 'max': 136.0, 'min': 76.0, 'me...","{'mean': 46.125, 'max': 51.0, 'min': 40.0, 'me...","{'mean': 60.125, 'max': 65.0, 'min': 55.0, 'me...","{'mean': 16.6, 'max': 19.0, 'min': 16.0, 'medi...","{'mean': 95.4, 'max': 97.0, 'min': 93.0, 'medi...",0,0,71.0,33,M,29,5.0
7996,19816095,25270868,33367844,16568,2143-05-20 19:00:00,1,"{'mean': 74.0, 'max': 80.0, 'min': 71.0, 'medi...","{'mean': 124.8, 'max': 136.0, 'min': 107.0, 'm...","{'mean': 50.6, 'max': 55.0, 'min': 43.0, 'medi...","{'mean': 73.0, 'max': 78.0, 'min': 62.0, 'medi...","{'mean': 14.6, 'max': 19.0, 'min': 13.0, 'medi...","{'mean': 98.4, 'max': 100.0, 'min': 97.0, 'med...",0,0,30.0,86,F,86,7.0
7997,16798872,26693411,36126710,58396,2180-04-08 19:00:00,1,"{'mean': 73.0, 'max': 74.0, 'min': 72.0, 'medi...","{'mean': 122.33333333333333, 'max': 135.0, 'mi...","{'mean': 41.833333333333336, 'max': 46.0, 'min...","{'mean': 61.1, 'max': 68.0, 'min': 55.0, 'medi...","{'mean': 23.727272727272727, 'max': 24.0, 'min...","{'mean': 94.72727272727273, 'max': 96.0, 'min'...",0,0,73.0,68,M,68,9.0
7998,10084586,21898489,30484216,19803,2130-01-30 08:00:00,1,"{'mean': 56.55555555555556, 'max': 60.0, 'min'...","{'mean': 101.11111111111111, 'max': 111.0, 'mi...","{'mean': 43.0, 'max': 51.0, 'min': 32.0, 'medi...","{'mean': 61.111111111111114, 'max': 71.0, 'min...","{'mean': 16.11111111111111, 'max': 23.0, 'min'...","{'mean': 94.75, 'max': 97.0, 'min': 92.0, 'med...",0,0,84.0,67,F,66,8.0


# Preparing the Final Data for Modeling
What We Did:

- Dropped rows with null values to ensure data quality.
- Converted categorical variables (e.g., gender) into numerical format suitable for modeling.

In [15]:
# drop null values
final_df_cleaned = final_df.dropna()

# Convert the values in the gender column: "male" to 1 and "female" to 0
final_df_cleaned['gender'] = final_df_cleaned['gender'].map({'M': 1, 'F': 0})


display(final_df_cleaned)

,subject_id,hadm_id,stay_id,caregiver_id,charttime,label,220045,220050,220051,220052,220210,220277,vasopressin,ventilation,apsiii,age,gender,anchor_age,SOFA
0,12765745,24692465,38629592,21810,2147-07-20 23:00:00,0,"{'mean': 60.0, 'max': 73.0, 'min': 55.0, 'medi...","{'mean': 111.0, 'max': 117.0, 'min': 100.0, 'm...","{'mean': 44.0, 'max': 57.0, 'min': 39.0, 'medi...","{'mean': 64.6, 'max': 70.0, 'min': 60.0, 'medi...","{'mean': 18.2, 'max': 19.0, 'min': 18.0, 'medi...","{'mean': 99.6, 'max': 100.0, 'min': 99.0, 'med...",0,0,43.0,83,0,83,3.0
1,19345174,24168361,32217390,1867,2178-03-11 20:01:00,0,"{'mean': 70.57142857142857, 'max': 77.0, 'min'...","{'mean': 101.0, 'max': 119.0, 'min': 90.0, 'me...","{'mean': 51.0, 'max': 57.0, 'min': 47.0, 'medi...","{'mean': 66.0, 'max': 75.0, 'min': 61.0, 'medi...","{'mean': 11.714285714285714, 'max': 18.0, 'min...","{'mean': 99.14285714285714, 'max': 100.0, 'min...",0,0,64.0,46,0,46,9.0
2,15782910,24816970,39146812,12665,2177-06-15 23:00:00,0,"{'mean': 98.0, 'max': 104.0, 'min': 84.0, 'med...","{'mean': 110.8, 'max': 125.0, 'min': 98.0, 'me...","{'mean': 59.2, 'max': 72.0, 'min': 53.0, 'medi...","{'mean': 78.4, 'max': 92.0, 'min': 70.0, 'medi...","{'mean': 23.2, 'max': 27.0, 'min': 17.0, 'medi...","{'mean': 97.4, 'max': 98.0, 'min': 97.0, 'medi...",0,0,59.0,77,1,75,6.0
3,10545967,27498545,32767451,55621,2186-05-08 23:00:00,0,"{'mean': 85.0, 'max': 95.0, 'min': 76.0, 'medi...","{'mean': 149.5, 'max': 163.0, 'min': 132.0, 'm...","{'mean': 62.6, 'max': 77.0, 'min': 56.0, 'medi...","{'mean': 91.5, 'max': 98.0, 'min': 85.0, 'medi...","{'mean': 22.0, 'max': 32.0, 'min': 17.0, 'medi...","{'mean': 93.4, 'max': 96.0, 'min': 92.0, 'medi...",0,0,32.0,61,0,61,5.0
4,19458874,27624253,37193919,36239,2149-10-25 23:00:00,0,"{'mean': 71.8, 'max': 77.0, 'min': 69.0, 'medi...","{'mean': 111.2, 'max': 121.0, 'min': 101.0, 'm...","{'mean': 59.6, 'max': 61.0, 'min': 56.0, 'medi...","{'mean': 75.6, 'max': 79.0, 'min': 73.0, 'medi...","{'mean': 14.0, 'max': 16.0, 'min': 12.0, 'medi...","{'mean': 97.8, 'max': 98.0, 'min': 97.0, 'medi...",0,0,89.0,57,1,56,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,11474034,22412399,30615140,34084,2127-09-30 02:00:00,1,"{'mean': 66.2, 'max': 96.0, 'min': 56.0, 'medi...","{'mean': 94.75, 'max': 136.0, 'min': 76.0, 'me...","{'mean': 46.125, 'max': 51.0, 'min': 40.0, 'me...","{'mean': 60.125, 'max': 65.0, 'min': 55.0, 'me...","{'mean': 16.6, 'max': 19.0, 'min': 16.0, 'medi...","{'mean': 95.4, 'max': 97.0, 'min': 93.0, 'medi...",0,0,71.0,33,1,29,5.0
7996,19816095,25270868,33367844,16568,2143-05-20 19:00:00,1,"{'mean': 74.0, 'max': 80.0, 'min': 71.0, 'medi...","{'mean': 124.8, 'max': 136.0, 'min': 107.0, 'm...","{'mean': 50.6, 'max': 55.0, 'min': 43.0, 'medi...","{'mean': 73.0, 'max': 78.0, 'min': 62.0, 'medi...","{'mean': 14.6, 'max': 19.0, 'min': 13.0, 'medi...","{'mean': 98.4, 'max': 100.0, 'min': 97.0, 'med...",0,0,30.0,86,0,86,7.0
7997,16798872,26693411,36126710,58396,2180-04-08 19:00:00,1,"{'mean': 73.0, 'max': 74.0, 'min': 72.0, 'medi...","{'mean': 122.33333333333333, 'max': 135.0, 'mi...","{'mean': 41.833333333333336, 'max': 46.0, 'min...","{'mean': 61.1, 'max': 68.0, 'min': 55.0, 'medi...","{'mean': 23.727272727272727, 'max': 24.0, 'min...","{'mean': 94.72727272727273, 'max': 96.0, 'min'...",0,0,73.0,68,1,68,9.0
7998,10084586,21898489,30484216,19803,2130-01-30 08:00:00,1,"{'mean': 56.55555555555556, 'max': 60.0, 'min'...","{'mean': 101.11111111111111, 'max': 111.0, 'mi...","{'mean': 43.0, 'max': 51.0, 'min': 32.0, 'medi...","{'mean': 61.111111111111114, 'max': 71.0, 'min...","{'mean': 16.11111111111111, 'max': 23.0, 'min'...","{'mean': 94.75, 'max': 97.0, 'min': 92.0, 'med...",0,0,84.0,67,0,66,8.0


# Extracting Statistical Features for Each Lab Test


- Extracted 12 statistical features for each lab test measurement.
- These features include mean, max, min, median, standard deviation, skewness, kurtosis, percentiles, etc.
- Removed the original nested data structures after extracting the features.

In [16]:
# Extract 12 feature values for each lab test
#diastolic-220051  systolic-220050   Heart Rate-220045  SpO2-220277   map-220052   Respiratory Rate-220210
def extract_features(item_series):
    return pd.Series({
        'mean': item_series.get('mean'),
        'max': item_series.get('max'),
        'min': item_series.get('min'),
        'median': item_series.get('median'),
        'std': item_series.get('std'),
        'skewness': item_series.get('skewness'),
        'kurtosis': item_series.get('kurtosis'),
        'q75': item_series.get('q75'),
        'q25': item_series.get('q25'),
        'mad': item_series.get('mad'),
        'range': item_series.get('range'),
        'var': item_series.get('var')
    })


final_df_cleaned[['220051_mean', '220051_max', '220051_min', '220051_median', '220051_std', '220051_skewness',
                  '220051_kurtosis', '220051_q75', '220051_q25', '220051_mad', '220051_range', '220051_var']] = \
    final_df_cleaned[220051].apply(extract_features)


final_df_cleaned[['220050_mean', '220050_max', '220050_min', '220050_median', '220050_std', '220050_skewness',
                  '220050_kurtosis', '220050_q75', '220050_q25', '220050_mad', '220050_range', '220050_var']] = \
    final_df_cleaned[220050].apply(extract_features)


final_df_cleaned[['220045_mean', '220045_max', '220045_min', '220045_median', '220045_std', '220045_skewness',
                  '220045_kurtosis', '220045_q75', '220045_q25', '220045_mad', '220045_range', '220045_var']] = \
    final_df_cleaned[220045].apply(extract_features)


final_df_cleaned[['220277_mean', '220277_max', '220277_min', '220277_median', '220277_std', '220277_skewness',
                  '220277_kurtosis', '220277_q75', '220277_q25', '220277_mad', '220277_range', '220277_var']] = \
    final_df_cleaned[220277].apply(extract_features)


final_df_cleaned[['220052_mean', '220052_max', '220052_min', '220052_median', '220052_std', '220052_skewness',
                  '220052_kurtosis', '220052_q75', '220052_q25', '220052_mad', '220052_range', '220052_var']] = \
    final_df_cleaned[220052].apply(extract_features)

final_df_cleaned[['220210_mean', '220210_max', '220210_min', '220210_median', '220210_std', '220210_skewness',
                  '220210_kurtosis', '220210_q75', '220210_q25', '220210_mad', '220210_range', '220210_var']] = \
    final_df_cleaned[220210].apply(extract_features)


final_df_cleaned.drop(columns=[220051, 220050, 220045, 220277, 220052, 220210], inplace=True)


#print(final_df_cleaned)

final_df_cleaned = final_df_cleaned.dropna()


display(final_df_cleaned)

,subject_id,hadm_id,stay_id,caregiver_id,charttime,label,vasopressin,ventilation,apsiii,age,...,220210_min,220210_median,220210_std,220210_skewness,220210_kurtosis,220210_q75,220210_q25,220210_mad,220210_range,220210_var
0,12765745,24692465,38629592,21810,2147-07-20 23:00:00,0,0,0,43.0,83,...,18.0,18.0,0.447214,1.500000,0.250000,18.0,18.0,0.320000,1.0,0.200000
1,19345174,24168361,32217390,1867,2178-03-11 20:01:00,0,0,0,64.0,46,...,9.0,9.0,4.309458,0.925992,-1.112564,14.0,9.0,3.591837,9.0,18.571429
2,15782910,24816970,39146812,12665,2177-06-15 23:00:00,0,0,0,59.0,77,...,17.0,24.0,4.266146,-0.499375,-1.190043,27.0,21.0,3.360000,10.0,18.200000
3,10545967,27498545,32767451,55621,2186-05-08 23:00:00,0,0,0,32.0,61,...,17.0,21.0,5.830952,1.192743,-0.105320,21.0,19.0,4.000000,15.0,34.000000
4,19458874,27624253,37193919,36239,2149-10-25 23:00:00,0,0,0,89.0,57,...,12.0,14.0,1.414214,0.000000,-0.500000,14.0,14.0,0.800000,4.0,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,11474034,22412399,30615140,34084,2127-09-30 02:00:00,1,0,0,71.0,33,...,16.0,16.0,1.341641,1.500000,0.250000,16.0,16.0,0.960000,3.0,1.800000
7996,19816095,25270868,33367844,16568,2143-05-20 19:00:00,1,0,0,30.0,86,...,13.0,14.0,2.509980,1.353275,0.056311,14.0,13.0,1.760000,6.0,6.300000
7997,16798872,26693411,36126710,58396,2180-04-08 19:00:00,1,0,0,73.0,68,...,21.0,24.0,0.904534,-2.846050,6.100000,24.0,24.0,0.495868,3.0,0.818182
7998,10084586,21898489,30484216,19803,2130-01-30 08:00:00,1,0,0,84.0,67,...,12.0,15.0,3.295620,0.896468,0.129267,18.0,14.0,2.567901,11.0,10.861111


#Prepare Data

In [17]:
# Exclude non-feature columns
non_feature_columns = ['label', 'subject_id', 'hadm_id', 'stay_id', 'caregiver_id', 'charttime','anchor_age']

# Features (X) and target (y)
X = final_df_cleaned.drop(columns=non_feature_columns)
y = final_df_cleaned['label']

# Ensure there are no missing values
X = X.dropna()
y = y.loc[X.index]

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: (6502, 78)
Target shape: (6502,)


# Feature Selection using ReliefF

- apply the ReliefF algorithm to rank the features
- fs_relief.feature_importances_ contains the importance scores assigned to each feature by ReliefF.
- create a DataFrame to map each feature to its score.
- sort the features in descending order of importance.

In [18]:
!pip install skrebate

  Preparing metadata (setup.py) ... done
  Created wheel for skrebate: filename=skrebate-0.62-py3-none-any.whl size=29256 sha256=c50bd7bc023d788a4e837c586add841604554b6ba4fa2dce08ffe3260f5d5325
  Stored in directory: /root/.cache/pip/wheels/dd/67/40/683074a684607162bd0e34dcf7ccdfcab5861c3b2a83286f3a
Successfully built skrebate


In [19]:
from skrebate import ReliefF

# Initialize ReliefF
fs_relief = ReliefF(n_neighbors=100, n_features_to_select=X.shape[1])

# Fit the model
fs_relief.fit(X.values, y.values)

# Get feature scores
relief_scores = fs_relief.feature_importances_

# Create a DataFrame for easy viewing
relief_feature_scores = pd.DataFrame({'Feature': X.columns, 'ReliefF_Score': relief_scores})

# Sort the features by score
relief_feature_scores.sort_values(by='ReliefF_Score', ascending=False, inplace=True)

# Display the top features
print("Top features according to ReliefF:")
print(relief_feature_scores.head(20))

Top features according to ReliefF:
          Feature  ReliefF_Score
54    220052_mean       0.133566
62     220052_q25       0.128549
6     220051_mean       0.124049
56     220052_min       0.122193
57  220052_median       0.121370
14     220051_q25       0.119953
8      220051_min       0.117689
9   220051_median       0.116640
61     220052_q75       0.114567
13     220051_q75       0.110865
55     220052_max       0.096569
7      220051_max       0.094632
18    220050_mean       0.031305
26     220050_q25       0.029843
21  220050_median       0.028840
20     220050_min       0.028585
25     220050_q75       0.028439
19     220050_max       0.028020
64   220052_range       0.013514
58     220052_std       0.013117


# Feature Selection using Mutual Information

In [20]:
from sklearn.feature_selection import mutual_info_classif

# Compute mutual information scores
mi_scores = mutual_info_classif(X, y)

# Create a DataFrame for easy viewing
mi_feature_scores = pd.DataFrame({'Feature': X.columns, 'MI_Score': mi_scores})

# Sort the features by score
mi_feature_scores.sort_values(by='MI_Score', ascending=False, inplace=True)

# Display the top features
print("Top features according to Mutual Information:")
print(mi_feature_scores.head(20))

Top features according to Mutual Information:
          Feature  MI_Score
62     220052_q25  0.186149
54    220052_mean  0.183089
56     220052_min  0.175520
57  220052_median  0.168655
14     220051_q25  0.166494
61     220052_q75  0.162105
6     220051_mean  0.160747
9   220051_median  0.158259
8      220051_min  0.153452
13     220051_q75  0.142696
7      220051_max  0.110292
55     220052_max  0.105240
25     220050_q75  0.039473
19     220050_max  0.031176
26     220050_q25  0.030967
18    220050_mean  0.030011
21  220050_median  0.028974
20     220050_min  0.027720
2          apsiii  0.019676
16   220051_range  0.014406


# Feature Selection using Gini Index

use a Decision Tree Classifier to compute feature importances based on the Gini Index.

- The Decision Tree Classifier computes feature importances based on how much each feature decreases the impurity (Gini Index) in the tree.

In [21]:
from sklearn.tree import DecisionTreeClassifier

# Initialize Decision Tree Classifier
dt_clf = DecisionTreeClassifier(criterion='gini', random_state=42)

# Fit the model
dt_clf.fit(X, y)

# Get feature importances
gini_importances = dt_clf.feature_importances_

# Create a DataFrame for easy viewing
gini_feature_importances = pd.DataFrame({'Feature': X.columns, 'Gini_Importance': gini_importances})

# Sort the features by importance
gini_feature_importances.sort_values(by='Gini_Importance', ascending=False, inplace=True)

# Display the top features
print("Top features according to Gini Importance:")
print(gini_feature_importances.head(20))

Top features according to Gini Importance:
            Feature  Gini_Importance
62       220052_q25         0.301217
56       220052_min         0.043118
24  220050_kurtosis         0.022352
2            apsiii         0.021856
54      220052_mean         0.021396
23  220050_skewness         0.021364
11  220051_skewness         0.019113
36  220045_kurtosis         0.017806
35  220045_skewness         0.017700
12  220051_kurtosis         0.017046
3               age         0.016584
42      220277_mean         0.016436
71  220210_skewness         0.016423
72  220210_kurtosis         0.016369
6       220051_mean         0.016054
32       220045_min         0.014727
59  220052_skewness         0.013859
14       220051_q25         0.013327
60  220052_kurtosis         0.013252
19       220050_max         0.011950


# Combine Feature Scores

In [22]:
# Merge the feature scores on the 'Feature' column
feature_scores = relief_feature_scores.merge(mi_feature_scores, on='Feature')
feature_scores = feature_scores.merge(gini_feature_importances, on='Feature')

print("Combined Feature Scores:")
print(feature_scores.head())

Combined Feature Scores:
         Feature  ReliefF_Score  MI_Score  Gini_Importance
0    220052_mean       0.133566  0.183089         0.021396
1     220052_q25       0.128549  0.186149         0.301217
2    220051_mean       0.124049  0.160747         0.016054
3     220052_min       0.122193  0.175520         0.043118
4  220052_median       0.121370  0.168655         0.010354


normalize the scores to ensure they are on the same scale before combining them.

In [37]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
feature_scores[['ReliefF_Score', 'MI_Score', 'Gini_Importance']] = scaler.fit_transform(
    feature_scores[['ReliefF_Score', 'MI_Score', 'Gini_Importance']])

print("Normalized Feature Scores:")
print(feature_scores.head(10))

Normalized Feature Scores:
         Feature  ReliefF_Score  MI_Score  Gini_Importance  Combined_Score
0     220052_q25       0.962536  1.000000         1.000000        2.962536
1    220052_mean       1.000000  0.983564         0.068316        2.051880
2     220052_min       0.915072  0.942899         0.140641        1.998612
3  220052_median       0.908926  0.906024         0.031548        1.846499
4    220051_mean       0.928936  0.863540         0.050528        1.843005
5     220051_q25       0.898348  0.894415         0.041448        1.834211
6     220052_q75       0.858128  0.870834         0.020958        1.749920
7  220051_median       0.873606  0.850174         0.022219        1.745998
8     220051_min       0.881440  0.824350         0.034412        1.740202
9     220051_q75       0.830484  0.766569         0.027466        1.624520


compute the combined score by summing the normalized scores.

In [24]:
# Compute the combined score
feature_scores['Combined_Score'] = (feature_scores['ReliefF_Score'] +
                                    feature_scores['MI_Score'] +
                                    feature_scores['Gini_Importance'])

# Rank features based on the combined score
feature_scores.sort_values(by='Combined_Score', ascending=False, inplace=True)

# Reset index
feature_scores.reset_index(drop=True, inplace=True)


print("Top features according to Combined Score:")
print(feature_scores[['Feature', 'Combined_Score']].head(20))

Top features according to Combined Score:
          Feature  Combined_Score
0      220052_q25        2.962536
1     220052_mean        2.051880
2      220052_min        1.998612
3   220052_median        1.846499
4     220051_mean        1.843005
5      220051_q25        1.834211
6      220052_q75        1.749920
7   220051_median        1.745998
8      220051_min        1.740202
9      220051_q75        1.624520
10     220051_max        1.334774
11     220052_max        1.302342
12     220050_q75        0.433348
13    220050_mean        0.420062
14     220050_max        0.416208
15     220050_q25        0.412956
16  220050_median        0.399028
17     220050_min        0.389575
18         apsiii        0.240990
19            age        0.186502


In [25]:
# Calculate the number of features to select (e.g., top 39%)
num_features = int(len(feature_scores) * 0.61)

print(f"Number of features to select: {num_features}")

Number of features to select: 47


In [26]:
# Get the list of selected features
selected_features = feature_scores['Feature'].values[:num_features]

print("Selected features:")
print(selected_features)

Selected features:
['220052_q25' '220052_mean' '220052_min' '220052_median' '220051_mean'
 '220051_q25' '220052_q75' '220051_median' '220051_min' '220051_q75'
 '220051_max' '220052_max' '220050_q75' '220050_mean' '220050_max'
 '220050_q25' '220050_median' '220050_min' 'apsiii' 'age' '220051_range'
 '220045_median' '220051_mad' '220050_kurtosis' '220277_mean' '220052_std'
 '220052_skewness' '220050_range' '220050_std' '220052_mad' '220052_range'
 '220045_var' '220051_std' '220045_q75' '220052_var' '220210_std'
 '220210_max' '220045_min' '220277_std' '220050_skewness'
 '220045_kurtosis' '220051_kurtosis' '220045_skewness' '220210_mean'
 '220277_mad' '220045_range' '220210_range']


In [27]:
# Create a new DataFrame with selected features
X_selected = X[selected_features]

print("Shape of X_selected:", X_selected.shape)

Shape of X_selected: (6502, 47)


# Split the Data into Training and Testing Sets

In [28]:
from sklearn.model_selection import train_test_split

# Stratify ensures the class distribution remains balanced
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.3, random_state=42, stratify=y)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

Training set size: (4551, 47)
Test set size: (1951, 47)


# Train and Evaluate Models

## Logistic Regression

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Initialize the model
lr_clf = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
lr_clf.fit(X_train, y_train)

# Predict on test set
y_pred_lr = lr_clf.predict(X_test)
y_pred_prob_lr = lr_clf.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr)
recall_lr = recall_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr)
auc_lr = roc_auc_score(y_test, y_pred_prob_lr)

print("Logistic Regression Performance:")
print(f'Accuracy: {accuracy_lr:.3f}')
print(f'Precision: {precision_lr:.3f}')
print(f'Recall: {recall_lr:.3f}')
print(f'F1 Score: {f1_lr:.3f}')
print(f'AUC: {auc_lr:.3f}')

Logistic Regression Performance:
Accuracy: 0.770
Precision: 0.750
Recall: 0.807
F1 Score: 0.777
AUC: 0.850


## Random Forest

In [30]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_clf.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf_clf.predict(X_test)
y_pred_prob_rf = rf_clf.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)
auc_rf = roc_auc_score(y_test, y_pred_prob_rf)

print("Random Forest Performance:")
print(f'Accuracy: {accuracy_rf:.3f}')
print(f'Precision: {precision_rf:.3f}')
print(f'Recall: {recall_rf:.3f}')
print(f'F1 Score: {f1_rf:.3f}')
print(f'AUC: {auc_rf:.3f}')

Random Forest Performance:
Accuracy: 0.772
Precision: 0.772
Recall: 0.768
F1 Score: 0.770
AUC: 0.853


## XGBoost

In [31]:
import xgboost as xgb

# Initialize the model
xgb_clf = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Train the model
xgb_clf.fit(X_train, y_train)

# Predict on test set
y_pred_xgb = xgb_clf.predict(X_test)
y_pred_prob_xgb = xgb_clf.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)
auc_xgb = roc_auc_score(y_test, y_pred_prob_xgb)

print("XGBoost Performance:")
print(f'Accuracy: {accuracy_xgb:.3f}')
print(f'Precision: {precision_xgb:.3f}')
print(f'Recall: {recall_xgb:.3f}')
print(f'F1 Score: {f1_xgb:.3f}')
print(f'AUC: {auc_xgb:.3f}')

XGBoost Performance:
Accuracy: 0.752
Precision: 0.749
Recall: 0.756
F1 Score: 0.752
AUC: 0.839


## AdaBoost

In [32]:
from sklearn.ensemble import AdaBoostClassifier

# Initialize the model
ada_clf = AdaBoostClassifier(n_estimators=100, random_state=42)

# Train the model
ada_clf.fit(X_train, y_train)

# Predict on test set
y_pred_ada = ada_clf.predict(X_test)
y_pred_prob_ada = ada_clf.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy_ada = accuracy_score(y_test, y_pred_ada)
precision_ada = precision_score(y_test, y_pred_ada)
recall_ada = recall_score(y_test, y_pred_ada)
f1_ada = f1_score(y_test, y_pred_ada)
auc_ada = roc_auc_score(y_test, y_pred_prob_ada)

print("AdaBoost Performance:")
print(f'Accuracy: {accuracy_ada:.6f}')
print(f'Precision: {precision_ada:.6f}')
print(f'Recall: {recall_ada:.6f}')
print(f'F1 Score: {f1_ada:.6f}')
print(f'AUC: {auc_ada:.6f}')

AdaBoost Performance:
Accuracy: 0.757560
Precision: 0.754601
Recall: 0.760041
F1 Score: 0.757311
AUC: 0.846939


## Support Vector Machine (SVM)

In [33]:
from sklearn.svm import SVC

# Initialize the model
svm_clf = SVC(probability=True, random_state=42)

# Train the model
svm_clf.fit(X_train, y_train)

# Predict on test set
y_pred_svm = svm_clf.predict(X_test)
y_pred_prob_svm = svm_clf.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)
auc_svm = roc_auc_score(y_test, y_pred_prob_svm)

print("SVM Performance:")
print(f'Accuracy: {accuracy_svm:.6f}')
print(f'Precision: {precision_svm:.6f}')
print(f'Recall: {recall_svm:.6f}')
print(f'F1 Score: {f1_svm:.6f}')
print(f'AUC: {auc_svm:.6f}')

SVM Performance:
Accuracy: 0.765761
Precision: 0.744762
Recall: 0.805355
F1 Score: 0.773874
AUC: 0.854357


## Gradient Boosting Decision Trees (GBDT)

In [34]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize the model
gbdt_clf = GradientBoostingClassifier(random_state=42)

# Train the model
gbdt_clf.fit(X_train, y_train)

# Predict on test set
y_pred_gbdt = gbdt_clf.predict(X_test)
y_pred_prob_gbdt = gbdt_clf.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy_gbdt = accuracy_score(y_test, y_pred_gbdt)
precision_gbdt = precision_score(y_test, y_pred_gbdt)
recall_gbdt = recall_score(y_test, y_pred_gbdt)
f1_gbdt = f1_score(y_test, y_pred_gbdt)
auc_gbdt = roc_auc_score(y_test, y_pred_prob_gbdt)

print("GBDT Performance:")
print(f'Accuracy: {accuracy_gbdt:.6f}')
print(f'Precision: {precision_gbdt:.6f}')
print(f'Recall: {recall_gbdt:.6f}')
print(f'F1 Score: {f1_gbdt:.6f}')
print(f'AUC: {auc_gbdt:.6f}')

GBDT Performance:
Accuracy: 0.774987
Precision: 0.766533
Recall: 0.787848
F1 Score: 0.777044
AUC: 0.855138


Create a DataFrame for the New Models


In [35]:
import pandas as pd

# Create a dictionary with model names and their corresponding performance metrics
performance_data = {
    'Model': [
        'Logistic Regression',
        'Random Forest',
        'XGBoost',
        'AdaBoost',
        'SVM',
        'GBDT'
    ],
    'Accuracy': [
        accuracy_lr,
        accuracy_rf,
        accuracy_xgb,
        accuracy_ada,
        accuracy_svm,
        accuracy_gbdt
    ],
    'Precision': [
        precision_lr,
        precision_rf,
        precision_xgb,
        precision_ada,
        precision_svm,
        precision_gbdt
    ],
    'Recall': [
        recall_lr,
        recall_rf,
        recall_xgb,
        recall_ada,
        recall_svm,
        recall_gbdt
    ],

    'F1 Score': [f1_lr,f1_rf, f1_xgb, f1_ada,f1_svm, f1_gbdt
    ],

    'AUC': [ auc_lr,auc_rf, auc_xgb, auc_ada, auc_svm, auc_gbdt
    ]
}

# Create the DataFrame
performance_df = pd.DataFrame(performance_data)

# Display the performance comparison table
print("\nModel Performance Comparison:")
print(performance_df)


Model Performance Comparison:
                 Model  Accuracy  Precision    Recall  F1 Score       AUC
0  Logistic Regression  0.769862   0.749522  0.807415  0.777392  0.849660
1        Random Forest  0.771912   0.772257  0.768280  0.770263  0.852529
2              XGBoost  0.752435   0.748980  0.755922  0.752435  0.839420
3             AdaBoost  0.757560   0.754601  0.760041  0.757311  0.846939
4                  SVM  0.765761   0.744762  0.805355  0.773874  0.854357
5                 GBDT  0.774987   0.766533  0.787848  0.777044  0.855138


In [36]:
# Required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb

# Evaluate models before feature reduction
X_train_full, X_test_full, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)

# Train and evaluate models on full data before feature reduction
# Logistic Regression
lr_full = LogisticRegression(max_iter=1000, random_state=42)
lr_full.fit(X_train_full, y_train)
y_pred_lr_full = lr_full.predict(X_test_full)
y_pred_prob_lr_full = lr_full.predict_proba(X_test_full)[:, 1]

# Random Forest
rf_full = RandomForestClassifier(n_estimators=100, random_state=42)
rf_full.fit(X_train_full, y_train)
y_pred_rf_full = rf_full.predict(X_test_full)
y_pred_prob_rf_full = rf_full.predict_proba(X_test_full)[:, 1]

# XGBoost
xgb_full = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_full.fit(X_train_full, y_train)
y_pred_xgb_full = xgb_full.predict(X_test_full)
y_pred_prob_xgb_full = xgb_full.predict_proba(X_test_full)[:, 1]

# AdaBoost
ada_full = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_full.fit(X_train_full, y_train)
y_pred_ada_full = ada_full.predict(X_test_full)
y_pred_prob_ada_full = ada_full.predict_proba(X_test_full)[:, 1]

# SVM
svm_full = SVC(probability=True, random_state=42)
svm_full.fit(X_train_full, y_train)
y_pred_svm_full = svm_full.predict(X_test_full)
y_pred_prob_svm_full = svm_full.predict_proba(X_test_full)[:, 1]

# GBDT
gbdt_full = GradientBoostingClassifier(random_state=42)
gbdt_full.fit(X_train_full, y_train)
y_pred_gbdt_full = gbdt_full.predict(X_test_full)
y_pred_prob_gbdt_full = gbdt_full.predict_proba(X_test_full)[:, 1]

# Calculate metrics for each model before feature reduction
# Logistic Regression
accuracy_lr_full = accuracy_score(y_test, y_pred_lr_full)
precision_lr_full = precision_score(y_test, y_pred_lr_full)
recall_lr_full = recall_score(y_test, y_pred_lr_full)
f1_lr_full = f1_score(y_test, y_pred_lr_full)
auc_lr_full = roc_auc_score(y_test, y_pred_prob_lr_full)

# Random Forest
accuracy_rf_full = accuracy_score(y_test, y_pred_rf_full)
precision_rf_full = precision_score(y_test, y_pred_rf_full)
recall_rf_full = recall_score(y_test, y_pred_rf_full)
f1_rf_full = f1_score(y_test, y_pred_rf_full)
auc_rf_full = roc_auc_score(y_test, y_pred_prob_rf_full)

# XGBoost
accuracy_xgb_full = accuracy_score(y_test, y_pred_xgb_full)
precision_xgb_full = precision_score(y_test, y_pred_xgb_full)
recall_xgb_full = recall_score(y_test, y_pred_xgb_full)
f1_xgb_full = f1_score(y_test, y_pred_xgb_full)
auc_xgb_full = roc_auc_score(y_test, y_pred_prob_xgb_full)

# AdaBoost
accuracy_ada_full = accuracy_score(y_test, y_pred_ada_full)
precision_ada_full = precision_score(y_test, y_pred_ada_full)
recall_ada_full = recall_score(y_test, y_pred_ada_full)
f1_ada_full = f1_score(y_test, y_pred_ada_full)
auc_ada_full = roc_auc_score(y_test, y_pred_prob_ada_full)

# SVM
accuracy_svm_full = accuracy_score(y_test, y_pred_svm_full)
precision_svm_full = precision_score(y_test, y_pred_svm_full)
recall_svm_full = recall_score(y_test, y_pred_svm_full)
f1_svm_full = f1_score(y_test, y_pred_svm_full)
auc_svm_full = roc_auc_score(y_test, y_pred_prob_svm_full)

# GBDT
accuracy_gbdt_full = accuracy_score(y_test, y_pred_gbdt_full)
precision_gbdt_full = precision_score(y_test, y_pred_gbdt_full)
recall_gbdt_full = recall_score(y_test, y_pred_gbdt_full)
f1_gbdt_full = f1_score(y_test, y_pred_gbdt_full)
auc_gbdt_full = roc_auc_score(y_test, y_pred_prob_gbdt_full)

# Print formatted comparison table
print("Comparison of the prediction performance before and after feature reduction.")
print("-" * 100)
print("         Before feature reduction                           After feature reduction")
print("Methods  Num   PRE    REC    ACC    F1     AUC     Num     PRE      REC    ACC    F1     AUC")
print("-" * 100)

# Define feature counts
total_features = X.shape[1]  # Total features before reduction
selected_features = X_selected.shape[1]  # Total features after reduction

# Print data for each model
methods = ['GBDT', 'SVM', 'LR', 'XGB', 'RF', 'AdaBoost']
for method in methods:
    # Print model name
    print(f"{method:<8}", end="")

    # Print metrics before feature reduction
    print(f"{total_features:<6}", end="")

    if method == 'GBDT':
        print(f"{precision_gbdt_full:.3f}  {recall_gbdt_full:.3f}  {accuracy_gbdt_full:.3f}  {f1_gbdt_full:.3f}  {auc_gbdt_full:.3f}     ", end="")
        print(f"{selected_features:<6} {precision_gbdt:.3f}  {recall_gbdt:.3f}  {accuracy_gbdt:.3f}  {f1_gbdt:.3f}  {auc_gbdt:.3f}")
    elif method == 'SVM':
        print(f"{precision_svm_full:.3f}  {recall_svm_full:.3f}  {accuracy_svm_full:.3f}  {f1_svm_full:.3f}  {auc_svm_full:.3f}     ", end="")
        print(f"{selected_features:<6} {precision_svm:.3f}  {recall_svm:.3f}  {accuracy_svm:.3f}  {f1_svm:.3f}  {auc_svm:.3f}")
    elif method == 'LR':
        print(f"{precision_lr_full:.3f}  {recall_lr_full:.3f}  {accuracy_lr_full:.3f}  {f1_lr_full:.3f}  {auc_lr_full:.3f}     ", end="")
        print(f"{selected_features:<6} {precision_lr:.3f}  {recall_lr:.3f}  {accuracy_lr:.3f}  {f1_lr:.3f}  {auc_lr:.3f}")
    elif method == 'XGB':
        print(f"{precision_xgb_full:.3f}  {recall_xgb_full:.3f}  {accuracy_xgb_full:.3f}  {f1_xgb_full:.3f}  {auc_xgb_full:.3f}     ", end="")
        print(f"{selected_features:<6} {precision_xgb:.3f}  {recall_xgb:.3f}  {accuracy_xgb:.3f}  {f1_xgb:.3f}  {auc_xgb:.3f}")
    elif method == 'RF':
        print(f"{precision_rf_full:.3f}  {recall_rf_full:.3f}  {accuracy_rf_full:.3f}  {f1_rf_full:.3f}  {auc_rf_full:.3f}     ", end="")
        print(f"{selected_features:<6} {precision_rf:.3f}  {recall_rf:.3f}  {accuracy_rf:.3f}  {f1_rf:.3f}  {auc_rf:.3f}")
    elif method == 'AdaBoost':
        print(f"{precision_ada_full:.3f}  {recall_ada_full:.3f}  {accuracy_ada_full:.3f}  {f1_ada_full:.3f}  {auc_ada_full:.3f}     ", end="")
        print(f"{selected_features:<6} {precision_ada:.3f}  {recall_ada:.3f}  {accuracy_ada:.3f}  {f1_ada:.3f}  {auc_ada:.3f}")

print("-" * 100)

Comparison of the prediction performance before and after feature reduction.
----------------------------------------------------------------------------------------------------
         Before feature reduction                           After feature reduction
Methods  Num   PRE    REC    ACC    F1     AUC     Num     PRE      REC    ACC    F1     AUC
----------------------------------------------------------------------------------------------------
GBDT    78    0.764  0.777  0.769  0.770  0.854     47     0.767  0.788  0.775  0.777  0.855
SVM     78    0.738  0.820  0.766  0.777  0.853     47     0.745  0.805  0.766  0.774  0.854
LR      78    0.752  0.804  0.770  0.777  0.848     47     0.750  0.807  0.770  0.777  0.850
XGB     78    0.748  0.750  0.750  0.749  0.835     47     0.749  0.756  0.752  0.752  0.839
RF      78    0.764  0.774  0.769  0.769  0.852     47     0.772  0.768  0.772  0.770  0.853
AdaBoost78    0.764  0.765  0.765  0.764  0.838     47     0.755  0.760  0.758 